In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
data = pd.read_csv("Student_marks.csv")
data.head()

,Maths,Physics,Chemistry,Result
0,17,27,22,0
1,72,82,77,1
2,97,18,13,0
3,8,42,37,0
4,32,25,20,0


In [5]:
data.shape

(1000, 4)

In [6]:
X = data.iloc[:, 0:-1]
y = data.iloc[:, -1]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
input_size = 3
hidden_size = 60
num_classes = 1
batch_size = 10
num_epochs = 50
learning_rate = 0.001

In [10]:
class Traindata(Dataset):
    def __init__(self, X_tensor, y_tensor):
        self.X =X_tensor
        self.y = y_tensor
        
    def __getitem__(self, index):
        return (self.X[index], self.y[index])
    
    def __len__(self):
        return len(self.X)
    
train_data = Traindata(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train))


In [11]:
y_test = y_test.to_numpy(dtype= 'float32')

In [12]:
class Testdata(Dataset):
    def __init__(self, X_tensor, y_tensor):
        self.X =X_tensor
        self.y = y_tensor
        
    def __getitem__(self, index):
        return (self.X[index], self.y[index])
    
    def __len__(self):
        return len(self.X)
    
test_data = Testdata(torch.FloatTensor(X_test),
                     torch.FloatTensor(y_test))

In [13]:
train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size=batch_size, shuffle = False)

In [14]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
  
    def forward(self,x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [15]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [16]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i,(X_train, y_train) in enumerate(train_loader):
        X_train, y_train = X_train.to(device), y_train.to(device)
        
        
        # Forward Pass
        y_pred = model(X_train)
        loss = criterion(y_pred, y_train.unsqueeze(1))

        #Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(i+1) % 20 ==0:
            print('Epoch [{}/{}], step[{}/{}], loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        
       

Epoch [1/50], step[20/80], loss: 0.6736
Epoch [1/50], step[40/80], loss: 0.5824
Epoch [1/50], step[60/80], loss: 0.5322
Epoch [1/50], step[80/80], loss: 0.4795
Epoch [2/50], step[20/80], loss: 0.5274
Epoch [2/50], step[40/80], loss: 0.3874
Epoch [2/50], step[60/80], loss: 0.4303
Epoch [2/50], step[80/80], loss: 0.2509
Epoch [3/50], step[20/80], loss: 0.4179
Epoch [3/50], step[40/80], loss: 0.1846
Epoch [3/50], step[60/80], loss: 0.2920
Epoch [3/50], step[80/80], loss: 0.2709
Epoch [4/50], step[20/80], loss: 0.4243
Epoch [4/50], step[40/80], loss: 0.2454
Epoch [4/50], step[60/80], loss: 0.1962
Epoch [4/50], step[80/80], loss: 0.3931
Epoch [5/50], step[20/80], loss: 0.3725
Epoch [5/50], step[40/80], loss: 0.4685
Epoch [5/50], step[60/80], loss: 0.1824
Epoch [5/50], step[80/80], loss: 0.3097
Epoch [6/50], step[20/80], loss: 0.1580
Epoch [6/50], step[40/80], loss: 0.1659
Epoch [6/50], step[60/80], loss: 0.3024
Epoch [6/50], step[80/80], loss: 0.3985
Epoch [7/50], step[20/80], loss: 0.0757


In [17]:
model.eval()

with torch.no_grad():
    correct=0
    total=0
    for X_test, y_test in test_loader:
        X_test = X_test.to(device)
        y_test = y_test.to(device)
        outputs = model(X_test)
        _, predicted = torch.max(outputs.data, 1)
        total+= y_test.shape[0]
        correct+= (predicted ==y_test).sum().item()
        
    print('Accuracy of network: {}%'.format(100*correct/total))

Accuracy of network: 72.5%
